In [ ]:
## THIS MODEL USING TRANSFER LEARNING TO CONSTRUCT A CNN FOR FACE RECOGNITION

In [1]:
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import optimizers
from keras import regularizers
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
conv_base = tf.keras.applications.VGG16(weights = 'imagenet',
                 include_top=False,
                 input_shape=(72, 96, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 72, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 72, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 72, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 36, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 36, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 36, 48, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 24, 128)       0     

In [3]:
for i in range (10,19):
    (conv_base.layers)[i].trainable =  False

In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 72, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 72, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 72, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 36, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 36, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 36, 48, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 24, 128)       0     

In [5]:
print(len(conv_base.layers))

19


In [6]:
conv_base.layers[10]

In [7]:
inp = conv_base.input
out = conv_base.layers[9].output

In [8]:
from keras.models import Model
model2 = Model(inp, out)

In [9]:
cont = 0
for layer in model2.layers:
    cont = cont + 1
    if (cont >= 8 and cont < 11):
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 72, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 72, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 72, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 36, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 36, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 36, 48, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 24, 128)       0     

In [ ]:
# train/val/test split (NOT RUNNING ANYMORE)
import splitfolders
# split data 
# input_folder = "../image_from_video/thermal_image"
output_folder = "../data_for_vgg16"
splitfolders.ratio("../image_from_video/thermal_image", output_folder,
    seed=1337, ratio=(.6, .2, .2), group_prefix=None, move=False) # default values

In [ ]:
# pre-processing-data (reduce noise)
# Face detection for thermal image

from PIL import Image
from matplotlib import pyplot as plt
import cv2
import numpy as np
import os
import sys
import time

# crop face_image
def convert_rgb_to_gray(file_name):
    img2 = cv2.imread(file_name)
    dst = cv2.fastNlMeansDenoisingColored(img2,None,10,10,7,21)
    cv2.imwrite(file_name, dst)
    print(" Complete convert: " + file_name + "\n")

# make directory store all processed image
def data_processing(folder_names, path):
    for folder_name in folder_names:
        folder_path = path + folder_name
        file_names = os.listdir(folder_path)
        
        for file in file_names:
            image_dir = folder_path + "/" + file
            convert_rgb_to_gray(image_dir)

In [ ]:
path = "../data_for_vgg16/test/"
folder_names = os.listdir(path)
data_processing(folder_names, path)

In [11]:
#Copying images to training, validation, and test directories
import os, shutil
import numpy as np
#Path where my dataset is stored
base_dir = 'C:/Users/vuduc/data_for_vgg16/'


# Directories for the training, validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')
print(validation_dir)
print(train_dir)

C:/Users/vuduc/data_for_vgg16/val
C:/Users/vuduc/data_for_vgg16/train


In [12]:
from keras import layers
from keras import models

model = Sequential()
model.add(model2)
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(19, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 18, 24, 256)       1735488   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 9, 12, 256)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 9, 12, 256)       1024      
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 19)                525331    
                                                                 
Total params: 2,261,843
Trainable params: 2,001,171
Non-

In [13]:
#Compilation stage
from tensorflow.keras import optimizers
learning_rate = 1e-4
momentum = 0.9

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4, decay=1e-6), #Decrease learning rate
              metrics=['accuracy'])

In [14]:
#Using ImageDataGenerator to read images from directories

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # Target directory
        target_size=(72, 96), # All images are resized from 240x320 to 72x96
        batch_size= 32,
        color_mode='rgb',
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(72, 96),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical')

Found 4560 images belonging to 19 classes.
Found 1520 images belonging to 19 classes.


In [15]:
#Training and validation stages
history = model.fit(
      train_generator,
      epochs= 10,
      validation_data=validation_generator,
      validation_steps=30)

Epoch 1/10
143/143 [==============================] - 213s 1s/step - loss: 0.3224 - accuracy: 0.9081 - val_loss: 0.0355 - val_accuracy: 0.9917
Epoch 2/10
143/143 [==============================] - 220s 2s/step - loss: 0.0097 - accuracy: 0.9980 - val_loss: 0.0122 - val_accuracy: 0.9958
Epoch 3/10
143/143 [==============================] - 220s 2s/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 0.0074 - val_accuracy: 0.9990
Epoch 4/10
143/143 [==============================] - 222s 2s/step - loss: 2.1787e-04 - accuracy: 1.0000 - val_loss: 0.0091 - val_accuracy: 0.9958
Epoch 5/10
143/143 [==============================] - 222s 2s/step - loss: 1.2793e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 0.9990
Epoch 6/10
143/143 [==============================] - 227s 2s/step - loss: 2.8911e-04 - accuracy: 0.9998 - val_loss: 9.0042e-04 - val_accuracy: 1.0000
Epoch 7/10
143/143 [==============================] - 228s 2s/step - loss: 3.2088e-06 - accuracy: 1.0000 - val_loss: 0.0074 - 

In [ ]:
model.save("thermal_net.h5")

In [ ]:
#Plot of the loss and accuracy during training

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#Test stage
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(72, 96),
        batch_size=64,
        color_mode='rgb',
        class_mode='categorical')

start_time = time.time()
test_loss, test_acc = model.evaluate(test_generator, steps = 30)
end_time = time.time()
testing_time = end_time - start_time
print('test acc:', test_acc)
print('test loss:', test_loss)